In [1]:
# !pip install langchain
# !pip install openai -U
# !pip install nervaluate
# !pip install faiss
# !pip install tiktoken
#!pip install pandas

In [2]:
### THE OVERVIEW: 
# 1. Prompt in LLM.
# 2. Convert the receipt text structured data
# 3. Use labels and K-nearest neighbors (embeddings) to classify the vendor into vendor categories.
# 4. Classify the items into categories; propose your own .
# 5. Plot these analytics of vendors and items.



In [3]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
import json

In [4]:
prompt = """
Can you please convert this unstructured receipt text data into a structured JSON object which includes ReceiptInfo and ITEMS.
Note that sometimes the quantity for an item may be located next to the item name.
Text in the structured JSON should appear in the same format as the unstructured receipt.
Please just give the structured JSON object and nothing else.

Here is the schema for the JSON object:
{{
  "ReceiptInfo": {{
    "merchant": "(string value)",
    "address": "(string value)",
    "city": "(string value)",
    "state": "(string value)",
    "phoneNumber": "(string value)",
    "tax": "(float value)",
    "total": "(float value)",
    "receiptDate": "(string value)",
    "receiptTime": "(string value)",


    "ITEMS": [
      {{
        "description": "(string value)",
        "quantity": "(integer value)",
        "unitPrice": "(float value)",
        "totalPrice": "(float value)",
        "discountAmount": "(float value)"
      }}
    ]
  }}
}}


Examples:

Receipt:
Give us feedback @ <UNKNOWN>
Thank you! ID <UNKNOWN> <UNKNOWN>
Walmart
301-604-0180 Mgr:DEBRA
3549 RUSSETT GRN
LAUREL MD 20724
ST# 01985 OP# 009049 TE# 49 TR# 04345
<UNKNOWN> PRK SB 008265754333 F
1.98 X
BOOSTER
088439295372
19.98 N
VOIDED BANKCARD TRANSACTION
TERMINAL <UNKNOWN> SC010112
TRANSACTION NOT COMPLETE
07/04/23
16:19:53
<UNKNOWN> BANKCARD TRANSACTION
TERMINAL # SC010112
TRANSACTION <UNKNOWN> COMPLETE
07/04/23
16:20:01
SUBTOTAL
21.96
TAX 1
6.000 %
0.12
TOTAL
22.08
VISA TEND
22.08
ACCOUNT #
**** **** <UNKNOWN> 2130 F
APPROVAL # 00592D
REF # 318500762875
TRANS ID - 583185732192035
VALIDATION - B339
PAYMENT SERVICE - E
TERMINAL # SC010112
07/04/23
16:20:21
CHANGE DUE
0.00
# ITEMS SOLD 2
<UNKNOWN> 0069 6254 8220 4557 9869
Walmart
Become a member
Scan for free 30-day trial
Low Prices You Can <UNKNOWN> Every <UNKNOWN>
07/04/23
16:20:22
***CUSTOMER <UNKNOWN>

JSON:
{{
  "ReceiptInfo": {{
    "merchant": "Walmart",
    "address": "3549 RUSSETT GRN",
    "city": "LAUREL",
    "state": "MD",
    "phoneNumber": "301-604-0180",
    "tax": "0.12",
    "total": "22.08",
    "receiptDate": "07/04/23",
    "receiptTime": "4:20pm",


    "ITEMS": [
      {{
        "description": "PRK SB",
        "quantity": "1",
        "unitPrice": "1.98",
        "totalPrice": "1.98",
        "discountAmount": "0.00"
      }},
      {{
        "description": "BOOSTER",
        "quantity": "1",
        "unitPrice": "19.98",
        "totalPrice": "19.98",
        "discountAmount": "0.00"
      }}
    ]
  }}
}}

Receipt:
Unstructured Receipt Text Data:
203
BUY ONE GET ONE FREE QUARTER POUNDER
W/CHEESE OR EGG MCMUFFIN
Go to www.mcdvoice.com within 7 days
and tell <UNKNOWN> about your visit.
Validation Code:
Expires 30 days after receipt date.
Valid at participating US
Survey Code:
<UNKNOWN>
McDonald's Restaurant <UNKNOWN>
2915 E MANCA RD
<UNKNOWN> 96822
TEL# <UNKNOWN>
Thank You Valued Customer
KS# 2
08/19/2023 03:33 PM
<UNKNOWN>
Order 03
1 Happy Meal Ch Burger
5.89
1 Cheeseburger
ONLY Onions
ONLY Ketchup
1 Extra Kids Fry
1 Apple Juice
1 NERF
Subtotal
5.89
Tax
0.28
Take-Out Total
6.17
Cashless
6.17
Change
0.00
MER# 467782
CARD ISSUER
ACCOUNT
Visa SALE
<UNKNOWN> <UNKNOWN> 130
TRANSACTION AMOUNT
6.17
CONTACTLESS
AUTHORIZATION CODE - <UNKNOWN>
SEQ# 107204
AID: A0000000031010
McDonald's Restaurant
Sign up for MyMcDonald's rewards
to earn points on future visit

JSON:
{{
  "ReceiptInfo": {{
    "merchant": "McDonald's Restaurant",
    "address": "2915 E MANOA RD",
    "city": "Honolulu",
    "state": "HI",
    "phoneNumber": "",
    "tax": "0.28",
    "total": "6.17",
    "receiptDate": "08/19/2023",
    "receiptTime": "03:33 PM",


    "ITEMS": [
      {{
        "description": "1 Happy Meal Ch Burger",
        "quantity": "1",
        "unitPrice": "5.89",
        "totalPrice": "5.89",
        "discountAmount": "0.00"
      }}
    ]
  }}
}}

Receipt:
Unstructured Receipt Text Data:
H
<UNKNOWN>
MART
http://www.hmart.com
458 Keawe st
Honolulu, <UNKNOWN> 96813
TEL (808) 219-0924
Your Cashier was TIM
WANG DANGMYUN
4.99 B
K-T TOPPOKI
PC
6.99 B
OTG SOUP RICECAKE
7.99 B
HT VEGE DUMPLING
14.99 B
TAX
1.65
****
BALANCE
36.61
Discover Credit - C
ACCOUNT <UNKNOWN> ************ 1153
APPROVAL CODE: 01263R
SEQUENCE NUMBER: 22907
No CVM
Amount USD $36.61
CARD:Discover CREDIT XXXX1153 EMV
APPROVAL CODE : 01263R
AID A0000001523010
TVR : 0000008000
IAD <UNKNOWN>
TSI : E800
APPLICATION CRYPTOGRAM CB8F25799B68A2B
APPLICATION PREFERRED NAME Discover Cr
APPLICATION LABEL : Discover
TC CB8F25799B68A2B3
<UNKNOWN> 00
RespDate : 06112023
Resp Time 211509
<UNKNOWN>
TOTAL AMOUNT: $36.61
RESPONSE CODE: APPROVED
06/11/23 09:15pm 113 20
Discover
<UNKNOWN> 61
CHANGE
0.00
TOTAL NUMBER OF ITEMS SOLD =
4
06/11/23 09:15pm 81 20 363 113

JSON:
{{
  "ReceiptInfo": {{
    "merchant": "H MART",
    "address": "458 Keawe st",
    "city": "Honolulu",
    "state": "HI",
    "phoneNumber": "(808) 219-0924",
    "tax": "1.65",
    "total": "36.61",
    "receiptDate": "06/11/23",
    "receiptTime": "09:15pm",


    "ITEMS": [
      {{
        "description": "WANG DANGMYUN",
        "quantity": "1",
        "unitPrice": "4.99",
        "totalPrice": "4.99",
        "discountAmount": "0.00"
      }},
      {{
        "description": "K-T TOPPOKI PC",
        "quantity": "1",
        "unitPrice": "6.99",
        "totalPrice": "6.99",
        "discountAmount": "0.00"
      }},
      {{
        "description": "OTG SOUP RICECAKE",
        "quantity": "1",
        "unitPrice": "7.99",
        "totalPrice": "7.99",
        "discountAmount": "0.00"
      }},
      {{
        "description": "HT VEGE DUMPLING",
        "quantity": "1",
        "unitPrice": "14.99",
        "totalPrice": "14.99",
        "discountAmount": "0.00"
      }}
    ]
  }}
}}

Receipt:
SAFEWAY
O
<UNKNOWN>
Store 204 Dir Kaipo Ah Mook Sang
Main (808) 988-2058
2855 East Manoa Road
HONOLULU HI 96822
GROCERY
6.99 B
MARINARA SCE
Regular Price
8.49
Member Savings
1.50-
REFRIG/FROZEN
LUC CHES COLBY JAC.
10.99 B
Regular Price
13.49
Member Savings
2.50-
MARIE CALLENDER'S
4.00 B
Regular Price
6.49
Member Savings
2.49-
MEAT
AIDELLS MEATBALLS
5.99 B
Regular Price
8.99
Member Savings
3.00-
TAX
1.32
**** BALANCE
29.29
Credit Purchase 08/01/23 20:15
CARD # <UNKNOWN>
<UNKNOWN> 861544421300 AUTH: 0006375D
PAYMENT AMOUNT
29.29
AL VISA CREDIT
AID A0000000031010
TVR 0000000000
TSI 0000
Visa
29.29
CHANGE
0.00
TOTAL NUMBER OF ITEMS SOLD =
4
08/01/23 20:15 204 51 151
8851
Gameplays Earned 2
Sweepstakes Entries Earned 2
*Subject to limit of 10 <UNKNOWN> day
Earn gameplays and sweepstakes entries
by shopping, then play
Flavor Adventure for a chance to win!
Scan the QR code to Play. Win. Save.
REWARDS AVAILABLE
2
POINTS EARNED TODAY
Base Points 27
Total 27
Points Towards Next Reward 57 of 100
YOUR CASHIER TODAY WAS SELF
YOUR SAVINGS
Member Savings
Total
9.49
Total Savings Value
9.49
25%
Thank <UNKNOWN> you for shopping <UNKNOWN>
For SAFEWAY FOR U questions
877-276-9637 or Safeway.com/foru call

JSON:
{{
  "ReceiptInfo": {{
    "merchant": "SAFEWAY",
    "address": "2855 East Manoa Road",
    "city": "Honolulu",
    "state": "HI",
    "phoneNumber": "(808) 988-2058",
    "tax": "1.32",
    "total": "29.29",
    "receiptDate": "08/01/23",
    "receiptTime": "8:15pm",


    "ITEMS": [
      {{
        "description": "MARINARA SCE",
        "quantity": "1",
        "unitPrice": "8.49",
        "totalPrice": "6.99",
        "discountAmount": "1.50"
      }},
      {{
        "description": "LUC CHES COLBY JAC.",
        "quantity": "1",
        "unitPrice": "13.49",
        "totalPrice": "10.99",
        "discountAmount": "2.50"
      }},
      {{
        "description": "MARIE CALLENDER'S",
        "quantity": "1",
        "unitPrice": "6.49",
        "totalPrice": "4.00",
        "discountAmount": "2.49"
      }},
      {{
        "description": "AIDELLS MEATBALLS",
        "quantity": "1",
        "unitPrice": "8.99",
        "totalPrice": "5.99",
        "discountAmount": "3.00"
      }}
    ]
  }}
}}

Here is the Unstructured Receipt Text Data (Turn this into structured JSON):

Receipt: {receipt}
"""

In [5]:
import os
def get_receipts():
    """Gets unstructured receipt text from each receipt text file in the receipts directory."""
    folder_path = 'receipts/text'
    receipts = []
    
    # List all files in the folder
    files = os.listdir(folder_path)
    
    # Iterate through each file
    for file_name in files:
        file_path = os.path.join(folder_path, file_name)
    
        # Check if it's a text file
        if file_name.endswith('.txt'):
            with open(file_path, 'r') as file:
                # Read the content and append to the array
                text_content = file.read()
                receipts.append(text_content)
    
    return receipts

test_receipts = get_receipts()

In [6]:
## Getting output from LLM.
# Uses OpenAI model.
model = ChatOpenAI()

# Creates a template for the prompt.
prompt_template = PromptTemplate.from_template(prompt)

# Inserts the prompt into model and gets output.
chain = LLMChain(llm=model, prompt=prompt_template)
receipt_outputs = [chain.run(receipt=test_receipt) for test_receipt in test_receipts]

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-g0T0cjlZV6fIkVIq68Jf5Vx8 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}

In [ ]:
## Named Entity Recognition:
def find_span(text, entity_text):
    """Finds the start and end positions for a given entity_text"""
    start = text.find(entity_text)
    if start == -1:
        return None
    end = start + len(entity_text)
    return (start, end), entity_text

def convert_to_prodigy_spans(receipt_text, entities):
    """Structures the receipt data by finding the position spans of each field indicated in the Structured JSON Object."""
    text_vals = []
    entities = json.loads(entities)
    prodigy_data = []
    receipt_info = entities["ReceiptInfo"]

    # Gets the label and entity_text for each corresponding label and entity listed.
    for label, entity_text in [
        ("MERCHANT", str(receipt_info.get("merchant", ""))), 
        ("ADDRESS", str(receipt_info.get("address", ""))), 
        ("CITY", str(receipt_info.get("city", ""))), 
        ("STATE", str(receipt_info.get("state", ""))), 
        ("PHONE", str(receipt_info.get("phoneNumber", ""))), 
        ("TAX", str(receipt_info.get("tax", ""))), 
        ("TOTAL", str(receipt_info.get("total", ""))), 
        ("DATE", str(receipt_info.get("receiptDate", ""))) 
    ]:

        # Tries to add positions of enties for corresponding labels.
        try:
            span, text = find_span(receipt_text, entity_text)
            text_vals.append(text)

            if span:
                start, end = span
                prodigy_data.append({"start": start, "end": end, "label": label})
        except:
            pass

    # Process item-level entities
    for item in receipt_info["ITEMS"]:
        for label, entity_text in [ # add .get() to all fields? 
            ("ITEM_DESC", item["description"]),
            ("QTY", str(item["quantity"])),
            ("UNIT_PRICE", str(item["unitPrice"])),
            ("TOTAL_PRICE", str(item["totalPrice"])),
            ("DISCOUNT", str(item.get("discountAmount", "")))  # Discount might not always be present
        ]:
            if entity_text:  # Check if the entity text is not empty
                span = find_span(receipt_text, entity_text)
                if span:
                    start, end = span
                    prodigy_data.append({"start": start, "end": end, "label": label})

    return prodigy_data, text_vals

# Example usage

prodigy_spans_true, text_vals = convert_to_prodigy_spans(test_receipts[0], receipt_outputs[0])
#print(json.dumps(prodigy_spans_true, indent=2))

In [ ]:
# Converts all receipt_outputs to JSON.
json_objects = [json.loads(receipt_output) for receipt_output in receipt_outputs]

# Gets the vendor name for each receipt JSON object.
vendors = [receipt_json['ReceiptInfo']['merchant'] for receipt_json in json_objects]
embeddings_model = OpenAIEmbeddings()

# Gets embeddings for the vendors.
vector_db = embeddings_model.embed_documents(vendors)
len(vector_db), len(vector_db[0])

In [ ]:
categories = ["Grocery and Supermarkets", "Restaurants and Food Services", "Clothing and Apparel", 
              "Health and Beauty", "Electronics and Appliances", "Home and Garden", "Entertainment and Leisure"]

category_train_test_data = [
['Milk', 'Eggs', 'Bread', 'Chicken', 'Bananas', 'Tomatoes', 'Potatoes', 'Onions', 'Cheese', 'Pasta', 'Rice', 'Cereal', 'Yogurt', 'Apples', 'Oranges', 'Carrots', 'Lettuce', 'Broccoli', 'Ground beef', 'Toothpaste', 'Shampoo', 'Soap', 'Toilet paper', 'Paper towels', 'Laundry detergent', 'Dish soap', 'Coffee', 'Tea', 'Sugar', 'Flour', 'Cooking oil', 'Ketchup', 'Mustard', 'Mayonnaise', 'Salad dressing', 'Chips', 'Soda', 'Water', 'Juice', 'Ice cream', 'Frozen pizza', 'Frozen vegetables', 'Canned beans', 'Canned tomatoes', 'Peanut butter', 'Jelly', 'Bacon', 'Sausages', 'Ham', 'Turkey', 'Bottled water', 'Energy bars', 'Cookies', 'Crackers', 'Candles', 'Light bulbs', 'Trash bags', 'Aluminum foil', 'Plastic wrap', 'Ziplock bags', 'Tissues', 'Hand sanitizer', 'Dishwasher detergent', 'Alcohol', 'Wine', 'Beer', 'Cat food', 'Dog food', 'Pet litter', 'Napkins', 'Baby diapers', 'Baby wipes', 'Baby formula', 'Feminine hygiene products', 'Bandages', 'Pain relievers', 'Cough medicine', 'Vitamins', 'Cleaning supplies', 'Air freshener', 'Dental floss', 'Lawn bags', 'Batteries', 'Candles', 'Greeting cards', 'Magazines', 'Postage stamps', 'Gift wrap'],
["Grilled Chicken Salad", "Cheeseburger", "Vegetarian Pizza", "French Fries", "Soda", "Iced Tea", "Chocolate Cake", "Spaghetti Bolognese", "Caesar Salad", "Margarita Pizza",
"Pasta Primavera", "Chicken Alfredo", "Garlic Bread", "Fish and Chips", "Mango Smoothie", "Cappuccino", "Tomato Soup", "Chicken Wings", "Shrimp Scampi", "Onion Rings",
"Club Sandwich", "Coffee", "Brownie Sundae", "Vegetable Stir-Fry", "Lemonade", "Beef Tacos", "Pesto Pasta", "Hot Chocolate", "Mushroom Risotto", "Greek Salad", "Churros",
"Chicken Parmesan", "Mojito", "Spinach Dip", "Pho", "Beef and Broccoli", "Tiramisu", "Hamburger", "Fried Chicken", "Green Tea", "Caprese Salad", "Quesadilla", "Nachos",
"Banana Split", "Pad Thai", "Sushi Roll", "Chicken Quesadilla", "Pineapple Fried Rice", "Apple Pie", "Clam Chowder", "Bruschetta", "Calamari", "Pancakes", "Chicken Caesar Wrap",
"Peach Iced Tea", "Avocado Toast", "Lobster Roll", "Eggplant Parmesan", "Tuna Salad Sandwich", "Raspberry Lemonade", "Crab Cakes", "Veggie Burger", "Tomato Basil Soup", "Omelette",
"Cobb Salad", "Chicken Teriyaki", "Chocolate Mousse", "Pesto Chicken Sandwich", "Tofu Stir-Fry", "Caramel Macchiato", "Beef Burrito", "Coconut Shrimp", "Pumpkin Pie", "Buffalo Wings",
"Strawberry Shortcake", "Quinoa Salad", "Shrimp Po' Boy", "Blueberry Pancakes", "Cajun Chicken Pasta", "Ginger Ale", "Pulled Pork Sandwich", "Fruit Salad", "Chicken Satay", "Key Lime Pie",
"Beef Stroganoff", "Chocolate Chip Cookies", "Cranberry Juice", "Egg Fried Rice", "Baked Ziti", "Cucumber Salad", "Veggie Wrap", "Black Bean Soup", "Sweet Potato Fries", "Pesto Pizza"],
['T-shirt', 'Jeans', 'Sweater', 'Dress', 'Jacket', 'Shorts', 'Skirt', 'Blouse', 'Shirt', 'Hoodie', 'Leggings', 'Cap', 'Socks', 'Underwear', 'Scarf', 'Gloves', 'Coat', 'Tank top', 'Polo shirt', 'Cardigan', 'Hat', 'Belt', 'Tie', 'Blazer', 'Pants', 'Jumpsuit', 'Vest', 'Kimono', 'Pajamas', 'Raincoat', 'Sunglasses', 'Handbag', 'Backpack', 'Wallet', 'Watch', 'Bracelet', 'Necklace', 'Earrings', 'Ring', 'Headband', 'Tights', 'Swimwear', 'Sandals', 'Boots', 'Sneakers', 'High heels', 'Loafers', 'Slippers', 'Umbrella', 'Tote bag', 'Gym bag', 'Laundry bag', 'Luggage', 'Iron', 'Hanger', 'Laundry detergent', 'Fabric softener', 'Lint roller', 'Sewing kit', 'Clothing tags', 'Thread', 'Needles', 'Buttons', 'Zipper', 'Measuring tape', 'Safety pins', 'Ironing board', 'Clothing rack', 'Lint brush', 'Shoe polish', 'Insoles', 'Shoehorn', 'Shoe rack', 'Hosiery', 'Fabric spray', 'Lint trap', 'Drawer organizers', 'Garment bag', 'Clothing brush', 'Stain remover', 'Lint shaver', 'Shoe inserts', 'Collar stays', 'Clothing steamer', 'Shoe care kit', 'Clothing repair patches', 'Sewing machine'],
['Shampoo', 'Conditioner', 'Facial cleanser', 'Moisturizer', 'Sunscreen', 'Body wash', 'Hand soap', 'Deodorant', 'Toothpaste', 'Mouthwash', 'Floss', 'Hairbrush', 'Hair ties', 'Razors', 'Shaving cream', 'Body lotion', 'Face mask', 'Exfoliating scrub', 'Makeup remover', 'Cotton pads', 'Cotton swabs', 'Tissues', 'Nail polish', 'Nail polish remover', 'Nail file', 'Acne treatment', 'Anti-aging cream', 'Serum', 'Eye cream', 'Lip balm', 'Face wash', 'Cleansing wipes', 'Essential oils', 'Aromatherapy candles', 'Bath salts', 'Bubble bath', 'Massage oil', 'Hair serum', 'Hair mask', 'Hand cream', 'Foot cream', 'Foot scrub', 'Body scrub', 'Scented lotion', 'Perfume', 'Cologne', 'Hair gel', 'Hair mousse', 'Styling cream', 'Blow dryer', 'Flat iron', 'Curling iron', 'Makeup brushes', 'Foundation', 'Concealer', 'Powder', 'Blush', 'Eyeshadow', 'Eyeliner', 'Mascara', 'Lipstick', 'Lip gloss', 'Makeup setting spray', 'Makeup remover wipes', 'Facial toner', 'Tweezers', 'Eyebrow pencil', 'Hair color', 'Hair dye', 'Hair accessories', 'Face sunscreen', 'Body sunscreen', 'Insect repellent', 'Antibacterial wipes', 'Hand sanitizer', 'Lip scrub', 'Hair clips', 'Sleep mask', 'Earplugs', 'Blister pads', 'Foot powder', 'Dental night guard', 'Collagen supplement', 'Vitamin C serum', 'Aloe vera gel', 'Tea tree oil', 'Sulfate-free shampoo', 'Organic conditioner', 'Natural skincare set', 'Reusable makeup remover pads'],
['Smart TV', 'Refrigerator', 'Laptop', 'Washing Machine', 'Air Conditioner', 'Bluetooth Speaker', 'Microwave Oven', 'Coffee Maker', 'Toaster', 'Vacuum Cleaner', 'Gaming Console', 'Smartwatch', 'Digital Camera', 'Headphones', 'Printer', 'Tablet', 'Blender', 'Hair Dryer', 'Electric Shaver', 'Rice Cooker', 'Iron', 'Food Processor', 'Smart Home Hub', 'Security Camera', 'Wireless Router', 'External Hard Drive', 'USB Flash Drive', 'Power Strip', 'HDMI Cable', 'USB-C Cable', 'Wireless Mouse', 'Mechanical Keyboard', 'Monitor', 'Soundbar', 'Fitness Tracker', 'Digital Scale', 'Robot Vacuum', 'Air Purifier', 'Portable Charger', 'Surge Protector', 'In-Ear Earphones', 'Gaming Mouse', 'Graphics Card', 'Motherboard', 'CPU', 'RAM', 'SSD', 'External SSD', 'Wireless Earbuds', 'Digital Voice Recorder', 'Projector', 'Smart Bulbs', 'Smart Thermostat', 'Coffee Grinder', 'Instant Pot', 'Deep Fryer', 'Juicer', 'Cordless Phone', 'Curling Iron', 'Flat Iron', 'Wireless Charger', 'Gaming Headset', 'Blue Light Glasses', 'Webcam', 'Smart Doorbell', 'Drone', 'GPS Tracker', 'Home Theater System', 'Digital Photo Frame', 'Camera Lens', 'Tripod', 'Solar Charger', 'Electric Scooter', 'Smart Refrigerator', 'Multi-Cooker', 'Air Fryer', 'E-reader', 'Smart Glasses', 'Waterproof Bluetooth Speaker', 'Smart Mirror', 'Car Dash Cam', 'Smart Door Lock', 'VR Headset', 'Digital Drawing Tablet', 'Electric Grill', 'Cordless Vacuum', 'Smart Kitchen Scale', 'Sleep Tracker', 'Wireless Charging Pad', 'Smart Pet Feeder'],
['Flower pot', 'Mulch', 'Garden gloves', 'Pruning shears', 'Hose', 'Watering can', 'Seeds', 'Fertilizer', 'Outdoor furniture set', 'Lawn mower', 'Garden shovel', 'Patio umbrella', 'Plant fertilizer', 'Compost bin', 'Garden rake', 'Plant labels', 'Garden trowel', 'Garden hose reel', 'Garden kneeler', 'Bird feeder', 'Weed killer', 'Garden edging', 'Garden sprayer', 'Pest control spray', 'Garden cart', 'Garden stakes', 'Wind chimes', 'Outdoor lights', 'Garden twine', 'Watering wand', 'Garden trellis', 'Patio heater', 'Garden hose nozzle', 'Rain barrel', 'Outdoor cushions', 'Garden hose splitter', 'Grill cover', 'Chiminea', 'Garden scissors', 'Lawn edger', 'Deck box', 'Garden mulcher', 'Hanging planters', 'Garden soil', 'Pergola kit', 'Garden mesh', 'Garden fence', 'Plant stand', 'Garden bench', 'Garden tool set', 'Fire pit', 'Garden gnome', 'Solar lights', 'Garden lanterns', 'Leaf blower', 'Outdoor rug', 'Garden hat', 'Garden apron', 'Garden shoes', 'Water feature', 'Garden decor', 'Potting bench', 'Garden tool organizer', 'Garden thermometer', 'Outdoor clock', 'Garden umbrella stand', 'Beehive', 'Outdoor storage shed', 'Garden hose hanger', 'Garden compost', 'Garden stakes', 'Garden netting', 'Garden seat cushion', 'Garden fountain', 'Garden wagon', 'Garden clogs', 'Garden trug', 'Folding outdoor table', 'Garden pond kit', 'Garden windmill', 'Garden tool bag', 'Garden tool rack', 'Outdoor broom', 'Garden tool sharpener', 'Garden hat', 'Outdoor lantern', 'Garden seat pad', 'Garden knee pads', 'Garden kneeler', 'Garden hose holder', 'Garden tool caddy', 'Garden gloves', 'Garden kneeling pad', 'Garden cart wheels'],
['Movie tickets', 'Popcorn', 'Soda', 'Candy', 'Arcade games tokens', 'Bowling lane rental', 'Shoe rental', 'Mini golf fees', 'Concert tickets', 'Merchandise', 'VIP lounge access', 'Theme park admission', 'Ride tickets', 'Food and beverages', 'Photo souvenirs', 'Escape room fees', 'Paintball session', 'Karaoke room rental', 'Virtual reality experience', 'Laser tag session', 'Amusement park parking', 'Go-kart race fees', 'Water park admission', 'Snack bar purchases', 'VIP seating upgrade', 'Musical instrument rental', 'Museum entrance fee', 'Art gallery tickets', 'Tour guide fees', 'Historical site admission', 'Outdoor adventure fees', 'Zip-lining charges', 'Horseback riding fees', 'Boat rental', 'Fishing gear rental', 'Concession stand purchases', 'Skating rink rental', 'Ice skate rental', 'Snow tubing fees', 'Ski lift pass', 'Snowboard rental', 'Themed event tickets', 'Casino gaming chips', 'Comedy show tickets', 'VIP meet and greet passes', 'Concession stand snacks', 'Live performance tickets', 'Stage play admission', 'Merchandise souvenirs', 'VIP lounge access', 'VIP parking pass', 'Surfing lessons', 'Parasailing fees', 'Jet ski rental', 'Beach cabana rental', 'Scuba diving lessons', 'Snorkeling gear rental', 'Golf course fees', 'Golf club rental', 'Tennis court rental', 'Fitness class fees', 'Spa services', 'Sauna session', 'Hot tub rental', 'Massage therapy', 'Pool access fees', 'Water aerobics class', 'Fitness center membership', 'Yoga class fees', 'Personal training session', 'Aerial silk class', 'Indoor climbing wall fees', 'Trampoline park admission', 'Dance class fees', 'Cooking class fees', 'Wine tasting event tickets', 'Brewery tour fees', 'Paint and sip class', 'Pottery painting session', 'Craft workshop fees', 'Photography class', 'Film screening tickets', 'Video game tournament entry', 'Board game night cover charge', 'Escape room team-building package', 'Trivia night entry', 'Concert merchandise', 'Sports event tickets', 'Autograph signing fees', 'VIP box seat rental', 'Stadium parking pass', 'Sports memorabilia purchases', 'Gaming convention admission', 'Cosplay contest entry']
]






    



In [ ]:
# sum(Vendor Embeddings) + sum(Item1 Embeddings) + sum(Item2 Embeddings), ...
# sum(category embeddings)

embeddings_model = OpenAIEmbeddings()
category_embeddings = embeddings_model.embed_documents(categories)


len(category_embeddings), len(category_embeddings[0])

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_cls = KNeighborsClassifier(n_neighbors=25)

knn_cls.fit(X_train, y_train_5)